In [1]:
from IPython.core.display import display, HTML
import sys
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [77]:
from datetime import date, datetime
import numpy as np
import pandas as pd
import dataframe_image as dfi
# pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
#pd.set_option('display.max_rows', 300)
#pd.set_option('display.max_columns', 20)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger, Fetcher
import WUBRG
from WUBRG import get_color_identity
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils import get_next_17lands_update_time, get_prev_17lands_update_time, get_name_slice, get_color_slice, get_date_slice


TRGT_SET = 'DMU'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

# Initialization

In [3]:
data_manager = None
set_data = None
print(f'Available sets: {SETS}')
print(f'Target set: {TRGT_SET}')

Available sets: ['DMU', 'SNC', 'NEO', 'VOW', 'MID']
Target set: DMU


In [4]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Current Local Time:  2022-08-27 21:25:03.638892
Last 17Lands Update: 2022-08-27 02:00:00
Current UTC Time:    2022-08-27 23:55:03.638892
Next 17Lands Update: 2022-08-28 02:00:00


In [5]:
start = datetime.utcnow()

if set_data is None:
    set_data = SetManager(TRGT_SET)
set_data.check_for_updates()
card_dict = set_data.SET_METADATA.CARD_DICT

end = datetime.utcnow()
print(f"\n --- Data loaded in {end - start}.")

Loading set metadata for: DMU
Fetching data for set: DMU
Fetching card data for set: DMU
Done!

Checking for missing data for DMU PremierDraft...
DMU PremierDraft has no summary data to get!
DMU PremierDraft has no historic data to get!
Finished checking for missing data for DMU PremierDraft.

Checking for missing data for DMU TradDraft...
DMU TradDraft has no summary data to get!
DMU TradDraft has no historic data to get!
Finished checking for missing data for DMU TradDraft.

Checking for missing data for DMU QuickDraft...
DMU QuickDraft has no summary data to get!
DMU QuickDraft has no historic data to get!
Finished checking for missing data for DMU QuickDraft.


 --- Data loaded in 0:00:03.575022.


In [23]:
tier_to_rank = {
    "A+": 12,
    "A": 11,
    "A-": 10,
    "B+": 9,
    "B": 8,
    "B-": 7,
    "C+": 6,
    "C": 5,
    "C-": 4,
    "D+": 3,
    "D": 2,
    "D-": 1,
    "F": 0,
    "SB": None,
    "TBD": None
}

rank_to_tier = {v: k for k, v in tier_to_rank.items()}

In [48]:
def fetch_raw_data(url):
    if url.startswith("https://www.17lands.com/tier_list/"):
        guid = url.replace("https://www.17lands.com/tier_list/", "")
        url = f"https://www.17lands.com/card_tiers/data/{guid}"
    fetcher = Fetcher()
    raw_data = fetcher.fetch(url)
    return raw_data

raw_data = fetch_raw_data("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b")
raw_data

[{'card_id': 13293,
  'name': 'Karn, Living Legacy',
  'url': 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/8/e83b1b87-01d5-4624-8a2b-4b771e13e59c.jpg?1661025516',
  'url_back': '',
  'rarity': 'M',
  'color': 'C',
  'tier': 'F',
  'sort_key': 0,
  'comment': '',
  'types': ['legendary', 'karn', 'planeswalker'],
  'cmc': 4.0,
  'flags': {'sideboard': False, 'synergy': False, 'buildaround': False}},
 {'card_id': 13255,
  'name': 'Charismatic Vanguard',
  'url': 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/5/a51764fe-0d75-4cfa-a699-0d9e7ffb7843.jpg?1660844238',
  'url_back': '',
  'rarity': 'C',
  'color': 'W',
  'tier': 'C-',
  'sort_key': 5,
  'comment': '',
  'types': ['dwarf', 'creature', 'soldier'],
  'cmc': 3.0,
  'flags': {'sideboard': False, 'synergy': False, 'buildaround': False}},
 {'card_id': 13460,
  'name': 'Phyrexian Vivisector',
  'url': 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/5/65617a8f-4bd8-4edb-b5ec-e20b448

In [47]:
def parse_raw_data(raw_data):
    data = dict()
    for card_rating in raw_data:
        data[card_rating['name']] = {
            'card': card_dict[card_rating['name']], 
            'tier': card_rating['tier'], 
            'rank': tier_to_rank[card_rating['tier']], 
            'synergy': card_rating['flags']['synergy'], 
            'buildaround': card_rating['flags']['buildaround']
        }
    return data

data = parse_raw_data(raw_data)
data

{'Karn, Living Legacy': {'card': Karn, Living Legacy,
  'tier': 'F',
  'rank': 0,
  'synergy': False,
  'buildaround': False},
 'Charismatic Vanguard': {'card': Charismatic Vanguard,
  'tier': 'C-',
  'rank': 4,
  'synergy': False,
  'buildaround': False},
 'Phyrexian Vivisector': {'card': Phyrexian Vivisector,
  'tier': 'C',
  'rank': 5,
  'synergy': False,
  'buildaround': False},
 'Phyrexian Warhorse': {'card': Phyrexian Warhorse,
  'tier': 'D-',
  'rank': 1,
  'synergy': False,
  'buildaround': False},
 'Pilfer': {'card': Pilfer,
  'tier': 'D+',
  'rank': 3,
  'synergy': False,
  'buildaround': False},
 'The Raven Man': {'card': The Raven Man,
  'tier': 'C+',
  'rank': 6,
  'synergy': False,
  'buildaround': False},
 'Sengir Connoisseur': {'card': Sengir Connoisseur,
  'tier': 'D',
  'rank': 2,
  'synergy': False,
  'buildaround': False},
 'Shadow Prophecy': {'card': Shadow Prophecy,
  'tier': 'D+',
  'rank': 3,
  'synergy': False,
  'buildaround': False},
 'Shadow-Rite Priest': {'

In [45]:
def calc_avg_rank(data):
    avg_rank = dict()
    colors = {"W", "U", "B", "R", "G", "", "M"}
    rarities = {"C", "U", "R", "M"}

    for c in colors:
        avg_rank[c] = dict()
        for r in rarities:
            avg_rank[c][r] = {"rank": 0, "count": 0}

    for name in data:
        card = data[name]
        color = card['card'].COLOR_IDENTITY
        rarity = card['card'].RARITY

        if not color in colors:
            color = 'M'

        d = avg_rank[color][rarity]
        d["rank"] = d["rank"] + card["rank"]
        d["count"] = d["count"] + 1


    for c in colors:
        for r in rarities:
            if avg_rank[c][r]["count"] == 0:
                avg_rank[c][r]["avg"] = None
            else:
                avg_rank[c][r]["avg"] = round(avg_rank[c][r]["rank"] / avg_rank[c][r]["count"], 1)

    return avg_rank

calc_avg_rank(data)

{'': {'M': {'rank': 8, 'count': 3, 'avg': 2.7},
  'C': {'rank': 12, 'count': 6, 'avg': 2.0},
  'U': {'rank': 12, 'count': 5, 'avg': 2.4},
  'R': {'rank': 6, 'count': 3, 'avg': 2.0}},
 'M': {'M': {'rank': 51, 'count': 8, 'avg': 6.4},
  'C': {'rank': 156, 'count': 30, 'avg': 5.2},
  'U': {'rank': 219, 'count': 40, 'avg': 5.5},
  'R': {'rank': 129, 'count': 21, 'avg': 6.1}},
 'U': {'M': {'rank': 13, 'count': 2, 'avg': 6.5},
  'C': {'rank': 59, 'count': 13, 'avg': 4.5},
  'U': {'rank': 35, 'count': 7, 'avg': 5.0},
  'R': {'rank': 40, 'count': 7, 'avg': 5.7}},
 'G': {'M': {'rank': 16, 'count': 2, 'avg': 8.0},
  'C': {'rank': 47, 'count': 13, 'avg': 3.6},
  'U': {'rank': 35, 'count': 7, 'avg': 5.0},
  'R': {'rank': 46, 'count': 7, 'avg': 6.6}},
 'B': {'M': {'rank': 14, 'count': 2, 'avg': 7.0},
  'C': {'rank': 39, 'count': 13, 'avg': 3.0},
  'U': {'rank': 32, 'count': 7, 'avg': 4.6},
  'R': {'rank': 45, 'count': 7, 'avg': 6.4}},
 'R': {'M': {'rank': 18, 'count': 2, 'avg': 9.0},
  'C': {'rank'

In [90]:
def to_data_frame(data, col_name='Rank'):
    ranks = dict()
    for card in data:
        ranks[card] = data[card]['rank']

    return pd.DataFrame.from_dict(ranks, orient="index", columns=[col_name])

frame = to_data_frame(data)
frame

,Rank
"Karn, Living Legacy",0
Charismatic Vanguard,4
Phyrexian Vivisector,5
Phyrexian Warhorse,1
Pilfer,3
...,...
Gibbering Barricade,3
Knight of Dusk's Shadow,6
Liliana of the Veil,6
Monstrous War-Leech,1


In [111]:
def frame_from_url(url, name):
    raw_data = fetch_raw_data(url)
    data = parse_raw_data(raw_data)
    frame = to_data_frame(data)
    frame.index.name = name
    
    return frame

In [112]:
zac = frame_from_url("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b", 'Zac')
klarm = frame_from_url("https://www.17lands.com/tier_list/3078f70b9a0d415ebf9f555439f5aedb", 'Klarm')
phyrre = frame_from_url("https://www.17lands.com/tier_list/a1ae9a695abb4921866f7f4a9a22e21f", 'Phyrre')
nomad = frame_from_url("https://www.17lands.com/tier_list/19ee3198a63b499bb7c25e6ceaae00ba", 'Nomad')

In [162]:
def merge_rankings(frame_list):
    frame = frame_list[0].copy(True)
    names = list()
    
    for indiv in frame_list:
        name = indiv.index.name
        names.append(name)
        frame[name] = indiv['Rank'].astype('Int64')
        
    frame = frame.drop('Rank', axis=1)
    frame['mean'] = round(frame.mean(axis=1), 3)
    frame['max'] = frame.max(axis=1)
    frame['min'] = frame.min(axis=1)
    frame['range'] = frame['max'] - frame['min']
    frame.index.name = ''
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(frame['mean'] - frame[name])**2
    dist['dist'] = round(dist.mean(axis=1)**0.5, 1)
    frame['dist'] = dist['dist']

    return frame
    
pd.set_option('display.max_rows', 20)

frame = merge_rankings([zac, klarm, phyrre, nomad])
frame.mean()

Zac       5.084291
Klarm     5.544402
Phyrre    5.084291
Nomad     5.863281
mean      5.369414
max       6.639847
min       4.160920
range     2.478927
dist      0.992337
dtype: float64

In [163]:
pd.set_option('display.max_rows', 300)
frame.sort_values('mean')

,Zac,Klarm,Phyrre,Nomad,mean,max,min,range,dist
,,,,,,,,,
Urza Assembles the Titans,0,0,0,0,0.000,0.0,0.0,0.0,0.0
Inscribed Tablet,0,0,2,0,0.500,2.0,0.0,2.0,0.9
Stronghold Arena,2,2,0,0,1.000,2.0,0.0,2.0,1.0
Yotia Declares War,0,2,2,0,1.000,2.0,0.0,2.0,1.0
Broken Wings,1,<NA>,1,<NA>,1.000,1.0,1.0,0.0,0.0
Rundvelt Hordemaster,0,3,1,0,1.000,3.0,0.0,3.0,1.2
Chaotic Transformation,0,2,0,2,1.000,2.0,0.0,2.0,1.0
Vodalian Hexcatcher,1,3,1,0,1.250,3.0,0.0,3.0,1.1
Temporary Lockdown,0,3,1,<NA>,1.333,3.0,0.0,3.0,1.2


In [164]:
pd.set_option('display.max_rows', 300)
frame.sort_values('range', ascending=False)

,Zac,Klarm,Phyrre,Nomad,mean,max,min,range,dist
,,,,,,,,,
Vesuvan Duplimancy,1,2,0,8,2.750,8.0,0.0,8.0,3.1
Temporal Firestorm,8,10,3,10,7.750,10.0,3.0,7.0,2.9
Choking Miasma,2,2,2,9,3.750,9.0,2.0,7.0,3.0
The World Spell,6,1,0,0,1.750,6.0,0.0,6.0,2.5
Drag to the Bottom,9,7,3,9,7.000,9.0,3.0,6.0,2.4
Ertai Resurrected,8,8,3,9,7.000,9.0,3.0,6.0,2.3
Runic Shot,4,5,6,9,6.000,9.0,4.0,5.0,1.9
"Rulik Mons, Warren Chief",6,8,5,3,5.500,8.0,3.0,5.0,1.8
The Phasing of Zhalfir,3,2,2,7,3.500,7.0,2.0,5.0,2.1


In [165]:
pd.set_option('display.max_rows', 300)
frame.sort_values('dist', ascending=False)

,Zac,Klarm,Phyrre,Nomad,mean,max,min,range,dist
,,,,,,,,,
Vesuvan Duplimancy,1,2,0,8,2.750,8.0,0.0,8.0,3.1
Choking Miasma,2,2,2,9,3.750,9.0,2.0,7.0,3.0
Temporal Firestorm,8,10,3,10,7.750,10.0,3.0,7.0,2.9
The World Spell,6,1,0,0,1.750,6.0,0.0,6.0,2.5
Drag to the Bottom,9,7,3,9,7.000,9.0,3.0,6.0,2.4
Ertai Resurrected,8,8,3,9,7.000,9.0,3.0,6.0,2.3
"Zur, Eternal Schemer",4,6,1,1,3.000,6.0,1.0,5.0,2.1
The Phasing of Zhalfir,3,2,2,7,3.500,7.0,2.0,5.0,2.1
Deathbloom Gardener,2,6,6,2,4.000,6.0,2.0,4.0,2.0


In [117]:
frame.T[["Dragon Whelp", "Soaring Drake"]].T

,Zac,Klarm,Phyrre,Nomad,mean,max,min,diff
Zac,,,,,,,,
Dragon Whelp,3.0,6.0,6.0,8.0,5.75,8.0,3.0,5.0
Soaring Drake,4.0,5.0,5.0,6.0,5.00,6.0,4.0,2.0


In [72]:
def supplement_frame(frame):
    #TODO: Populate frame with information about Colours, Costs and Rarities.
    pass

frame = supplement_frame(data)
frame

,Rank
"Karn, Living Legacy",0
Charismatic Vanguard,4
Phyrexian Vivisector,5
Phyrexian Warhorse,1
Pilfer,3
...,...
Gibbering Barricade,3
Knight of Dusk's Shadow,6
Liliana of the Veil,6
Monstrous War-Leech,1


In [127]:
frame

,Zac,Klarm,Phyrre,Nomad,mean,max,min,diff
,,,,,,,,
"Karn, Living Legacy",0,3.0,4,3.0,2.50,4.0,0.0,4.0
Charismatic Vanguard,4,3.0,6,3.0,4.00,6.0,3.0,3.0
Phyrexian Vivisector,5,5.0,3,7.0,5.00,7.0,3.0,4.0
Phyrexian Warhorse,2,5.0,4,5.0,4.00,5.0,2.0,3.0
Pilfer,3,4.0,2,2.0,2.75,4.0,2.0,2.0
...,...,...,...,...,...,...,...,...
Gibbering Barricade,3,5.0,5,5.0,4.50,5.0,3.0,2.0
Knight of Dusk's Shadow,6,7.0,7,6.0,6.50,7.0,6.0,1.0
Liliana of the Veil,6,9.0,5,7.0,6.75,9.0,5.0,4.0


In [146]:
dist = frame.copy(True)
dist = dist.drop('mean', axis=1)
dist = dist.drop('max', axis=1)
dist = dist.drop('min', axis=1)
dist = dist.drop('diff', axis=1)
dist['Zac'] = abs(frame['mean'] - dist['Zac'])
dist['Klarm'] = abs(frame['mean'] - dist['Klarm'])
dist['Phyrre'] = abs(frame['mean'] - dist['Phyrre'])
dist['Nomad'] = abs(frame['mean'] - dist['Nomad'])
dist['dist'] = dist.mean(axis=1)
frame['dist'] = dist['dist']
dist

,Zac,Klarm,Phyrre,Nomad,dist
,,,,,
"Karn, Living Legacy",2.500000,0.500000,1.500000,0.50,1.000000
Charismatic Vanguard,0.000000,1.000000,2.000000,1.00,1.400000
Phyrexian Vivisector,0.000000,0.000000,2.000000,2.00,1.400000
Phyrexian Warhorse,2.000000,1.000000,0.000000,1.00,1.200000
Pilfer,0.250000,1.250000,0.750000,0.75,1.000000
The Raven Man,0.750000,0.750000,1.750000,3.25,1.700000
Sengir Connoisseur,1.000000,1.000000,0.000000,2.00,1.200000
Shadow Prophecy,1.000000,0.000000,0.000000,1.00,0.800000
Shadow-Rite Priest,1.500000,2.500000,0.500000,0.50,1.400000


In [147]:
pd.set_option('display.max_rows', 300)
frame.sort_values('dist', ascending=False)

,Zac,Klarm,Phyrre,Nomad,mean,max,min,diff,dist
,,,,,,,,,
Temporal Firestorm,8,10.0,3,10.0,7.750000,10.0,3.0,7.0,2.500000
Choking Miasma,2,2.0,2,9.0,3.750000,9.0,2.0,7.0,2.500000
Sol'Kanar the Tainted,10,9.0,5,9.0,8.250000,10.0,5.0,5.0,2.300000
Ertai Resurrected,8,8.0,3,9.0,7.000000,9.0,3.0,6.0,2.200000
Drag to the Bottom,9,7.0,3,9.0,7.000000,9.0,3.0,6.0,2.200000
Urborg Lhurgoyf,4,4.0,7,8.0,5.750000,8.0,4.0,4.0,2.200000
"Raff, Weatherlight Stalwart",5,5.0,8,9.0,6.750000,9.0,5.0,4.0,2.200000
"Shanna, Purifying Blade",6,9.0,4,4.0,5.750000,9.0,4.0,5.0,2.200000
Vesuvan Duplimancy,1,2.0,0,8.0,2.750000,8.0,0.0,8.0,2.100000
